In [1]:
import pandas as pd

In [2]:
from nsepy import get_history as gh
import datetime as dt

In [3]:
start = dt.datetime(2013,6,1)
end = dt.datetime(2022,2,11)
stk_data = gh(symbol='TATACOFFEE',start=start,end=end)


In [4]:
stk_data=stk_data[["Open","High","Low","Close"]]
#stk_data.to_csv("Tatacoffee13_21.csv")

In [7]:
#column="Close"

In [5]:
from sklearn.preprocessing import MinMaxScaler
Ms = MinMaxScaler()
data1= Ms.fit_transform(stk_data)
print("Len:",data1.shape)

Len: (177, 4)


In [6]:
data1=pd.DataFrame(data1,columns=["Open","High","Low","Close"])

In [7]:
training_size = round(len(data1 ) * 0.80)
print(training_size)
X_train=data1[:training_size]
X_test=data1[training_size:]
print("X_train length:",X_train.shape)
print("X_test length:",X_test.shape)
y_train=data1[:training_size]
y_test=data1[training_size:]
print("y_train length:",y_train.shape)
print("y_test length:",y_test.shape)

142
X_train length: (142, 4)
X_test length: (35, 4)
y_train length: (142, 4)
y_test length: (35, 4)


In [8]:
import warnings
warnings.filterwarnings("ignore")

In [9]:
performance={"Model":[],"RMSE":[],"MaPe":[],"Lag":[],"Test":[]}

In [10]:
def cominbation(dataset,listt):
    print(listt)
    datasetTwo=dataset[listt]
    test_obs = 28
    train =datasetTwo[:-test_obs]
    test = datasetTwo[-test_obs:]
    from statsmodels.tsa.api import VAR
    for i in [1,2,3,4,5,6,7,8,9,10]:
        model = VAR(train)
        results = model.fit(i)
        print('Order =', i)
        print('AIC: ', results.aic)
        print('BIC: ', results.bic)
        print()
    x = model.select_order(maxlags=12)
    order=x.selected_orders["aic"]
    result = model.fit(order)
    #result.summary()
    lagged_Values = train.values[-order:]
    pred = result.forecast(y=lagged_Values,steps=28) 
    preds=pd.DataFrame(pred,columns=listt)
    preds.to_csv("varforecasted_{}.csv".format(test_obs))
    from sklearn.metrics import mean_squared_error
    rmse= round(mean_squared_error(test,pred,squared=False))
    from sklearn.metrics import mean_absolute_percentage_error
    mape=mean_absolute_percentage_error(test,pred)
    performance["Model"].append(listt)
    performance["RMSE"].append(rmse)
    performance["MaPe"].append(mape)
    performance["Lag"].append(order)
    performance["Test"].append(test_obs)
    perf=pd.DataFrame(performance)
    return perf,result,pred

In [18]:
listt=["Close","High","Open","Low"]
#listt=["AQI_calculated","PM10","PM2.5","NOx","NO2","NO","NH3","SO2","CO",'year']


In [19]:
perf,result,pred=cominbation(data1,listt)

['Close', 'High', 'Open', 'Low']
Order = 1
AIC:  -26.35793582045063
BIC:  -25.952907134806832

Order = 2
AIC:  -26.23103202876293
BIC:  -25.49868119118446

Order = 3
AIC:  -26.068132183966977
BIC:  -25.00547777075579

Order = 4
AIC:  -26.004242578592297
BIC:  -24.608257099388165

Order = 5
AIC:  -25.925452519076494
BIC:  -24.193061427657163

Order = 6
AIC:  -25.812982019038294
BIC:  -23.741062697178222

Order = 7
AIC:  -25.685484014227164
BIC:  -23.270864727735994

Order = 8
AIC:  -25.492416894076122
BIC:  -22.73187572010507

Order = 9
AIC:  -25.344561102070948
BIC:  -22.23482482674111

Order = 10
AIC:  -25.440206782579512
BIC:  -21.977949767950495



In [20]:
data1

,Open,High,Low,Close
0,0.140240,0.072381,0.073518,0.021463
1,0.049331,0.000000,0.064032,0.017488
2,0.096547,0.038730,0.120158,0.060413
3,0.110641,0.028571,0.090909,0.023052
4,0.089500,0.012698,0.079051,0.023052
...,...,...,...,...
172,0.613108,0.503492,0.667194,0.616057
173,0.591261,0.480635,0.633992,0.601749
174,0.633545,0.510476,0.687747,0.626391
175,0.633545,0.537778,0.695652,0.662162


In [21]:
perf

,Model,RMSE,MaPe,Lag,Test
0,"[Close, High]",0.0,0.114329,2,28
1,"[Close, High, Open]",0.0,0.111524,1,28
2,"[Close, High, Open, Low]",0.0,0.118421,1,28
